# Datenbanken

Zur Verwaltung von Daten kann man neben einfachen *Dateisystemen* auch **Datenbanksysteme** (**DBS**) verwenden. 
Datenbanksysteme bestehen aus einem **Datenbankmanagementsystem** (**DBMS**) und einer **Datenbank** (**DB**) dahinter.
Das DBMS ist ein Programm, das auf dem eigenen Computer oder einem Server läuft.
Am weitesten verbreitet sind **relationale DBS**, bei denen die Daten in **Tabellenform** gespeichert werden.
Bekannte Vertreter relationaler DBS sind etwa *Microsoft SQL Server* oder *MySQL*.

Eine externe Anwendung kommuniziert mit dem DBMS, indem sie **Anfragen** (*queries*) an dieses übermittelt und ggf. **Ergebnismengen** (*result sets*) als Antwort bekommt.
Die Anwendung kann die Datenbank selbst nicht sehen, der Zugriff erfolgt immer über das DBMS als Mittler (*Kapselung* [*Encapsulation*] bzw. *Information Hiding*).

Bei relationalen DBS werden Anfragen in der **Structured Query Language** (kurz: **SQL**) formuliert und als String an das DBMS übertragen. 
Diese Sprache ist grundsätzlich unabhängig vom jeweiligen DBS, wenngleich die meisten DBS nur eine Auswahl aller möglichen SQL-Befehle unterstützen.
Mit SQL lässt sich das **Schema** der Datenbank definieren, indem man Tabellen erstellt und die Namen und **Datentypen** der Spalten festlegt.
Außerdem stellt SQL Befehle zur Verfügung, um Datensätze anzulegen (*Create*), strukturiert abzufragen (*Read*), zu verändern (*Update*) und zu löschen (*Delete*). 
Man spricht auch von **CRUD**-Befehlen. 

Wenn die Daten in verschiedenen Spalten einer Tabelle voneinander abhängen, kann es zu **Anomalien** beim Anlegen, Aktualisieren und Löschen von Datensätzen kommen (*Create-*, *Update-* und *Delete-Anomalien*).
Um solche Anomalien in relationalen Datenbanken zu vermeiden, verteilt man die Daten bei der Modellierung auf unterschiedliche Tabellen, die miteinander verknüpft sind.
Man spricht dabei auch von **Normalisierung** und unterschiedlichen **Normalformen**.
Für die Verknüpfung der Tabellen benötigt man Felder (Attribute), die einen Datensatz eindeutig identifizieren. Diese Felder heißen **Schlüssel** bzw. **Schlüsselkandidaten**.
In jeder Tabelle muss mindestens eine Spalte als **Primärschlüssel** (*Primary Key*) ausgezeichnet werden. 
Setzt sich ein Schlüssel aus mehreren Spalten einer Tabelle zusammen, so spricht man von einem *zusammengesetzten Schlüssel*. 

**SQLite** ist eine Programmbibliothek, die ein vollwertiges DBMS in eigene Anwendungen integriert, ohne dass dafür zusätzliche Software installiert werden muss.

## SQLite in Python
Eine umfassende Erklärung mit zahlreichen Beispielen findet sich in der [Dokumentation](https://docs.python.org/3.6/library/sqlite3.html) des Python-Moduls. 
Die offizielle [SQLite Dokumentation](https://www.sqlite.org/docs.html) ist demgegenüber relativ unübersichtlich.

### Normale Verwendung

In [ ]:
import sqlite3
# Einbinden der Bibliothek

conn = sqlite3.connect('database.db')
# Verbindung zu einer Datenbankdatei herstellen
# Datei wird neu angelegt, wenn sie noch nicht existiert
# Befehl liefert ein Connection-Objekt zurück

cursor = conn.cursor()   
# Liefert ein Cursor-Objekt zurück, das auf eine Ergebnismenge zeigt
# Auf dem Cursor-Objekt lassen sich SQL-Befehle ausführen

cursor.execute('''
    CREATE TABLE IF NOT EXISTS tabelle ( 
        spalte1 TEXT PRIMARY KEY,
        spalte2 TEXT);''')
# Führt einen SQL-Befehl aus, der als String angegeben wird

In [ ]:
# Sauberes Escaping (verhindert SQL-Injection)

mydict = {"eins" : "wert1", "zwei" : "wert2"}
cursor.execute("INSERT INTO tabelle VALUES (:eins, :zwei)", mydict)

mytuple = ("wert1", "wert2")
cursor.execute("INSERT INTO tabelle VALUES (?, ?)", mytuple)

In [ ]:
conn.commit()
# Macht Schreibvorgänge dauerhaft - erst danach stehen sie anderen Verbindungen zur Verfügung
# Zurücksetzen per conn.rollback()

conn.close()
# Beendet die Verbindung

### Cell- und Line-Magic

Erweiterung laden und Verbindung zur Datenbank herstellen:

In [ ]:
%load_ext sql
%sql sqlite:///database.db

Befehl als Cell Magic (mehrzeilige Anfragen möglich):

In [ ]:
%%sql
SELECT spalte1, spalte2 FROM tabelle

Befehl als Line Magic (kann als rechte Seite einer Zuweisung geschrieben werden, sodass man das Ergebnis einer Variablen zuordnen kann):

In [ ]:
%sql SELECT spalte1, spalte2 FROM tabelle

## SQL

Tabellen erstellen (`CREATE`) und ändern (`ALTER`):

In [ ]:
%%sql
CREATE TABLE tabelle (
    spalte1 TEXT,
    spalte2 TEXT
);
ALTER TABLE tabelle ADD COLUMN spalte3 TEXT;

Daten einfügen: `INSERT`

In [ ]:
%sql INSERT INTO tabelle (spalte1, spalte2) VALUES ("wert1", "wert2")
%sql INSERT INTO tabelle (spalte2, spalte1) VALUES ("wert2", "wert1")
%sql INSERT INTO tabelle (spalte1) VALUES ("wert1")
%sql INSERT INTO tabelle VALUES ("wert1", "wert2")

Daten bearbeiten: `UPDATE`

In [ ]:
%sql UPDATE tabelle SET spalte1 = "test"
# Achtung: Ändert alle Datensätze!
%sql UPDATE tabelle SET spalte1 = "neuerwert1" WHERE wert1 = "wert1"
# Ändert alle Spalten in tabelle, auf die die WHERE-Bedingung zutrifft

Daten löschen: `DELETE`

In [ ]:
%sql DELETE FROM tabelle
# Achtung: Löscht alle Datensätze in der Tabelle!
%sql DELETE FROM tabelle WHERE wert1 = "neuerwert1"
# Löscht alle Datensätze aus tabelle, auf die die WHERE-Bedingung zutrifft

Daten abfragen: `SELECT`

In [ ]:
# Abfragespalten, Bedingung, Sortierung
# ----------------------------------------

%sql SELECT * FROM tabelle
%sql SELECT spalte1 FROM tabelle

%sql SELECT spalte1, spalte2 FROM tabelle WHERE wert1 = "neuerwert1"
# Mehrere Bedingungen können mit AND / OR kombiniert werden

%sql SELECT * FROM tabelle ORDER BY spalte2 DESC
%sql SELECT * FROM tabelle ORDER BY spalte1 ASC, spalte2 DESC
# Sortierung kann mit ASC (ascending, aufsteigend) oder DESC (descending, absteigend) festgelegt werden


# Aggregatfunktionen
# ----------------------------------------

%sql SELECT COUNT(*) FROM tabelle
%sql SELECT MAX(spalte2) FROM tabelle

%sql SELECT spalte1, COUNT(*) FROM tabelle GROUP BY spalte1
# Aggregatfunktion wird auf die gesamte Gruppe angewendet

%sql SELECT spalte1, COUNT(*) AS zaehler FROM tabelle WHERE spalte1 = "neuerwert1" GROUP BY spalte1 HAVING zaehler > 5
# AS weist dem Feld eine bestimmte Bezeichnung zu
# WHERE wird vor der Gruppierung als Filter angewendet
# HAVING bezieht sich auf das gruppierte Ergebnis


# Verknüpfung von Tabellen
# ----------------------------------------
% sql SELECT buecher.titel, autoren.name FROM buecher LEFT JOIN autoren ON buecher.autorenID = autoren.autorenID
# LEFT JOIN füllt die Spalte autoren.name mit NULL, falls kein Datensatz gefunden wird

Zugriff auf Ergebnisse in Python:

In [ ]:
cursor.execute("SELECT * FROM tabelle")

resultlist = cursor.fetchall()
# gibt eine Liste mit allen Datensätzen als Tupel zurück

In [ ]:
cursor.execute("SELECT * FROM tabelle")

result = cursor.fetchone()
# gibt den Datensatz zurück, auf den das Cursor-Objekt gerade zeigt
# das Cursor-Objekt zeigt dann auf den nächsten Datensatz

In [ ]:
cursor.execute("SELECT * FROM tabelle")
for row in cursor:     # Der cursor ist iterable
    # do something
    print row